In [16]:
import pandas as pd
import numpy as np
import pymongo
from hmmlearn.hmm import CategoricalHMM

from sklearn.model_selection import train_test_split


In [17]:
import warnings

warnings.filterwarnings(action="ignore")

In [18]:
# data 기본 경로
ABSOLUTE_PATH = "C:\\Users\\rudnf\\vscode\\Graduation\\final\\data\\"
SEASONS = ["spring", "summer", "fall", "winter"]
KS = [25, 50, 75, 100]

MONGO_URI = "mongodb://localhost:27017/"

In [19]:
def season_match(season):
    months = []
    if season == "spring":
        months = ["03", "04", "05"]
    if season == "summer":
        months = ["06", "07", "08"]
    if season == "fall":
        months = ["09", "10", "11"]
    if season == "winter":
        months = ["12", "01", "02"]
    return months

In [20]:
import ast
def df_str_to_list(df):
    try:
        columns = ['trajectory_id', 'POI_sequence', 'time_period_sequence']
        for column in columns:
            df[column] = df[column].apply(ast.literal_eval)
        return df
    except (ValueError, SyntaxError):
        print("Invalid input string format.")
        

In [21]:
def label_sequence(POI_sequence, time_period_sequence):
    l_sequence = []
    for idx in range(0, len(POI_sequence)):
        LID = int(POI_sequence[idx].replace('POI', ''))
        T = 0
        
        if time_period_sequence[idx] == 'dawn':
            T = 0
        elif time_period_sequence[idx] == 'morning':
            T = 1
        if time_period_sequence[idx] == 'afternoon':
            T = 2
        elif time_period_sequence[idx] == 'night':
            T = 3
            
        l_sequence.append( LID * 4 + T )
    return l_sequence

In [22]:
#****************** initial probability **********************#
def calculate_initial_probability(X, n_components, smoothing_factor=1.0):
    start_count = np.zeros(n_components)
    start_proba = np.zeros(n_components)

    # start_point
    for value in X[:, 1]:
        start_count[int(value.replace("POI", ""))] += 1

    total = np.sum(start_count)
    
    ## Laplace 스무딩적용
    start_proba = (start_count + smoothing_factor) / (total + n_components)
    return start_proba


In [23]:
# #****************** transition probability **********************#
# # transition_matrix : transition 횟수 count 저장, transition_probability : transition 확률 저장
def calculate_transition_probability(X, n_components, smoothing_factor=1.0):
    
    transition_count = np.zeros((n_components, n_components))  
    transition_proba = np.zeros((n_components, n_components))
    
    # transition 횟수 count
    # X[:, 2] : 'path'
    for trajectory in X[:, 3]:

        for i in range(len(trajectory) - 1):
            cur_POI_num = int(trajectory[i].replace('POI',''))
            next_POI_num = int(trajectory[i+1].replace('POI',''))
            
            transition_count[cur_POI_num, next_POI_num] += 1
            

    for i in range(len(transition_count)):
        total = np.sum(transition_count[i])
        
        ## Laplace 스무딩적용
        transition_proba[i] = (transition_count[i] + smoothing_factor) / (total + n_components)
    return transition_proba


In [24]:
#****************** emission probability **********************#
# 각 POI일때 label sequence가 나올확률
def calculate_emission_probability(X, n_components, n_features, smoothing_factor=1.0):
        # emission Probability 구하기
    emission_count = np.zeros(shape=(n_components, n_features))
    emission_proba = np.zeros(shape=(n_components, n_features))
 
    # X[:, [2,4]] : ['POI_sequence','l_sequence']
    for p, l_seq in X[:, [3,5]]:
        for cur_POI_num, cur_l_seq in zip(p, l_seq):
            cur_POI_num = int(cur_POI_num.replace('POI',''))
            emission_count[cur_POI_num][cur_l_seq] +=1
        

    for i in range(len(emission_count)):
        total = np.sum(emission_count[i])
        
         ## Laplace 스무딩적용
        emission_proba[i] = (emission_count[i] + smoothing_factor) / (total + n_features)

    return emission_proba


In [25]:
def _get_collection_data(mongo_uri, db_name, collection_name):
    client = pymongo.MongoClient(mongo_uri)

    db = client[db_name]
    collection = db[collection_name]

    data = list(collection.find({}, {"_id": 0}))

    client.close()

    return data


In [26]:
def get_trajectory(season, k):
    db_name = f"{season}"

    months = season_match(season)
    
    all_data = []
    for month in months :    
        collection_trajectory_name = f"trajectory_{month}_cluster_{k}"
        data = _get_collection_data(MONGO_URI, db_name, collection_trajectory_name)
        all_data.extend(data)
    return pd.DataFrame(all_data)
    

# Top 3 

In [27]:
def get_next_top3_POI(hmm_model, user_trajectory):
    
    X = np.array(user_trajectory)

    # X_test의 한 row에 대해 0~N_COMPONENTS까지 추가한 새로운 row들
    # sequences는 한 X_test 의 row에 대해 0 ~ N_COMPONENTS까지 추가한 sequence이기 때문에 모두 길이가 같다
    can_sequences = [np.append(X, i) for i in range(hmm_model.n_features)]


    # HMM viterbi 과정
    # (Log likelihood, Hidden state sequence) return 
    decodes = [hmm_model.decode(seq.reshape(-1,1)) for seq in can_sequences]

    sorted_decode = sorted(decodes, key=lambda x:x[0])[::-1]

    # 순서대로 1, 2, 3
    unique_top_3_indices = []
    for log_likelihood, hidden_state_seq in sorted_decode:
        recommendation_POI = hidden_state_seq[-1]
        if recommendation_POI not in unique_top_3_indices:
            unique_top_3_indices.append(recommendation_POI)
        
        if len(unique_top_3_indices) == 3:
            break
    return unique_top_3_indices


In [28]:
def top_n_accuracy(correct, predict, n):
    correct_at_n = sum(1 if c in predict[i][:n] else 0 for i, c in enumerate(correct))
    top_n_accuracy = correct_at_n / len(correct)
    return top_n_accuracy

In [30]:
seasonal_result_df = pd.DataFrame(columns=['season', 'K', 'Top N accuracy'])

idx =0
for season in SEASONS:
    for k in KS:      
        print(f"Season : {season}, K : {k}")

       
        trajectory = get_trajectory(season, k)
        trajectory = df_str_to_list(trajectory)
        trajectory['l_sequence'] = [np.array(label_sequence(POI_sequence, time_period_sequence)) for POI_sequence, time_period_sequence in trajectory[['POI_sequence', 'time_period_sequence']].values]


        select_columns = ['trajectory_id', 'start_point', 'end_point', 'POI_sequence', 'time_period_sequence', 'l_sequence']
        X = np.array(trajectory[select_columns])

        X_train, X_test = train_test_split(X, test_size=0.3)

        # time_period 수
        TIME_PERIOD = 4
        # cluster 수 : Hidden State수
        n_components = k
        n_features = n_components * TIME_PERIOD


        # 모델 불러오기
        hmm_model = CategoricalHMM(n_components=n_components, n_features=n_features)
    

        hmm_model.startprob_ = calculate_initial_probability(X_train, n_components)
        hmm_model.transmat_ = calculate_transition_probability(X_train, n_components)
        hmm_model.emissionprob_ = calculate_emission_probability(X_train, n_components, n_features)

        predict_next_point = []
        current_path_last_point = []


        for seq in X_test:
            removed_last_point_seq = seq[5][:-1]

            current_path_last_point.append(seq[5][-1]//4)
            
            next_point = get_next_top3_POI(hmm_model, removed_last_point_seq)

            predict_next_point.append(next_point)
            
        n_value = 3
        top_N = top_n_accuracy(current_path_last_point, predict_next_point, n_value)

        print("Top-N Accuracy:", top_N)

        seasonal_result_df.loc[idx] = [season, k, top_N]
        idx += 1



Season : spring, K : 25
Top-N Accuracy: 0.7971900826446281


ValueError: cannot set a row with mismatched columns

In [ ]:
seasonal_result_df

In [ ]:
# seasonal_result_df.to_csv("seasonal_top3.csv", index=False)